# Bluetooth Low Energy

## Requirements
- adb
- pyshark / tshark

## 1. Data acquisition
### Android HCI logs
Retrieve HCI logs from an Android device with `adb`:

In [76]:
import subprocess
import datetime

# possible locations and names of the HCI logs file
btsnoop_hci_log_files = [
    {
        'path': '/sdcard/',
        'file_name': 'btsnoop_hci.log'
    },
    {
        'path': '/sdcard/Android/data/',
        'file_name': 'btsnoop_hci.log'
    }
]

btsnoop_hci_dst_folder = 'hci_logs/'
btsnoop_hci_dst_last_file_name = 'hci_log_last.pcap'
btsnoop_hci_dst_file_name = 'hci_log.pcap'

for btsnoop_hci_log_file in btsnoop_hci_log_files:
    try:
        cmd = 'adb pull ' + btsnoop_hci_log_file['path'] + btsnoop_hci_log_file['file_name']  +  ' ' + btsnoop_hci_dst_folder
        subprocess.check_output(cmd, stderr=subprocess.STDOUT, shell=True)
        print('Found: ' + btsnoop_hci_path)
        cmd = 'mv ' + btsnoop_hci_dst_folder + btsnoop_hci_log_file["file_name"] + ' ' + btsnoop_hci_dst_folder + btsnoop_hci_dst_file_name
        subprocess.check_output(cmd, stderr=subprocess.STDOUT, shell=True)
        print('Moved to: ' + btsnoop_hci_dst_folder + btsnoop_hci_dst_last_file_name)
        d = datetime.datetime.today().strftime('%Y%m%d_%H%M')
        cmd = 'cp ' + btsnoop_hci_dst_folder + btsnoop_hci_dst_last_file_name + ' ' + btsnoop_hci_dst_folder + d + '_' + btsnoop_hci_dst_file_name
        subprocess.check_output(cmd, stderr=subprocess.STDOUT, shell=True)
        print('Backed up as: ' + btsnoop_hci_dst_folder + d + '_' + btsnoop_hci_dst_file_name)
        break
    except subprocess.CalledProcessError:
        print('Not found: ' + btsnoop_hci_log_file['path'] + btsnoop_hci_log_file['file_name'])
        pass

Found: /sdcard/btsnoop_hci.log
Moved to: hci_logs/hci_log_last.pcap
Backed up as: hci_logs/20200220_1420_hci_log.pcap


### Live capture with Sniffle

### Load an existing capture

## 2. Data pre-processing

Parse all BLE packets and store only the following BLE ATT packets:
- Read Response
- Write Request
- Write Command
- Value Notification
- Value Indication

In [120]:
#!/usr/bin/env python3

import pyshark
import binascii

file_name = 'hci_logs/hci_log_last.pcap'

opcodes = {
    0x0b: 'Read Response',
    0x12: 'Write Request',
    0x52: 'Write Command',
    0x1b: 'Value Notification',
    0x1d: 'Value Indication'
}

att_packets=[]

def parse_packet(packet):
    if 'BTATT' in str(packet.layers):
        try:
            att_packet = {}
            att_packet['opcode'] = opcodes.get(int(packet['BTATT'].opcode, 16), 'Unknown')
            att_packet['handle'] = int(packet['BTATT'].handle, 16)
            att_packet['data'] = str(packet['BTATT'].value).replace(':', '')
            #att_packet['data'] = binascii.hexlify(str(packet['BTATT'].value).replace(':', ''))
            #att_packet['data_len'] = len(att_packet['data'])
            att_packets.append(att_packet)
        except AttributeError:
            #print('AttributeError')
            return
        
try:
    capture = pyshark.FileCapture(file_name)
    await capture.packets_from_tshark(parse_packet)

except FileNotFoundError:
    print('File not found: ' + file_name)

if len(att_packets) == 0:
    print('No ATT packets found')
else:
    print('Found: %d ATT packets' % len(att_packets))
    %store att_packets

Found: 57 ATT packets
Stored 'att_packets' (list)


## 3. Data analysis

In [63]:
#!/usr/bin/env python3

from scapy.all import *
import binascii

file_name = 'hci_logs/20200220_1354_hci_log.pcap'

att_packets = []

def parse_packet(packet):
    packet = BTLE(packet.load)
    
    if packet.haslayer(ATT_Hdr):
        att_packet = {}
        if packet.haslayer(ATT_Write_Request): 
            att_packet['opcode'] = 'Write Request'
            att_packet['gatt_handle'] = packet.getlayer(ATT_Write_Request).gatt_handle
            att_packet['data'] = binascii.hexlify(packet.getlayer(ATT_Write_Request).data)
            att_packets.append(att_packet)
        elif packet.haslayer(ATT_Write_Command): 
            att_packet['opcode'] = 'Write Command'
            att_packet['gatt_handle'] = packet.getlayer(ATT_Write_Command).gatt_handle
            att_packet['data'] = binascii.hexlify(packet.getlayer(ATT_Write_Command).data)
            att_packets.append(att_packet)
        elif packet.haslayer(ATT_Handle_Value_Indication): 
            att_packet['opcode'] = 'Value Indication'
            att_packet['gatt_handle'] = packet.getlayer(ATT_Handle_Value_Indication).gatt_handle
            att_packet['data'] = binascii.hexlify(packet.getlayer(ATT_Handle_Value_Indication).value)
            att_packets.append(att_packet)
        elif packet.haslayer(ATT_Handle_Value_Notification): 
            att_packet['opcode'] = 'Value Notification'
            att_packet['gatt_handle'] = packet.getlayer(ATT_Handle_Value_Notification).gatt_handle
            att_packet['data'] = binascii.hexlify(packet.getlayer(ATT_Handle_Value_Notification).value)
            att_packets.append(att_packet)
        elif packet.haslayer(ATT_Read_Request): 
            print(packet.show())
        

with PcapReader(file_name) as pcap_reader:
    for packet in pcap_reader:
        parse_packet(packet)
        #parse_packet(BTLE(packet.load))

%store att_packets

Scapy_Exception: Not a pcapng capture file (bad magic: b'btsn')

In [121]:
import pandas as pd

#pd.set_option('display.max_colwidth', None)

df = pd.DataFrame(att_packets)
df = df[['opcode', 'handle', 'data']]

df

,opcode,handle,data
0,Write Request,12,4b0114002d00
1,Value Notification,14,4b01140025000000
2,Write Command,22,10000c90040001ce001e000302e0fd8e
3,Value Notification,24,2c0028910080000050414143585330302d303034
4,Value Notification,24,2d5230315d100000000000000000001200000000
5,Value Notification,24,0200fa85
6,Write Command,22,0e000a8004000006200000f0fe5c
7,Value Notification,24,10000c8000800008200000f000f0fcec
8,Value Notification,24,0e000a9300804e3032453031fd8d
9,Write Command,22,090005e6040031fee0


In [118]:
#import pandas as pd

df = pd.DataFrame(att_packets)
df = df[['opcode', 'handle', 'data']]

def highlight_by_handle(row):
    if row.opcode == 'Write Request':
        return ['background-color: green; color: white']*3
    elif row.opcode == 'Value Indication':
        return ['background-color: yellow']*3

df.style.apply(highlight_by_handle, axis=1)

AttributeError: 'NoneType' object has no attribute 'rstrip'

In [108]:
df

,opcode,handle,data
0,Write Request,12,4b0114002d00
1,Value Notification,14,4b01140025000000
2,Write Command,22,10000c90040001ce001e000302e0fd8e
3,Value Notification,24,2c0028910080000050414143585330302d303034
4,Value Notification,24,2d5230315d100000000000000000001200000000
5,Value Notification,24,0200fa85
6,Write Command,22,0e000a8004000006200000f0fe5c
7,Value Notification,24,10000c8000800008200000f000f0fcec
8,Value Notification,24,0e000a9300804e3032453031fd8d
9,Write Command,22,090005e6040031fee0
